In [2]:
import torch
import XRDXRFutils as xutils
from glob import glob
from src import nets, dataset
import yaml
from torch.utils.data import Dataset,DataLoader,WeightedRandomSampler
import numpy as np
from matplotlib import pyplot as plt
import tifffile

In [3]:
# load the model
base = './run/synth_scales0,5-1,5_uboundonly_CNN28_ch_run_0/'
model_path = glob(base+'*.pth')[-1]
print(model_path)
with open(base+'config.yaml','r') as file:
    config = yaml.load(file,Loader=yaml.FullLoader)
model = nets.CNN2(channels=config['channels'], n_outputs=len(config['labels']))

./run/synth_scales0,5-1,5_uboundonly_CNN28_ch_run_0\0099.pth


In [4]:
#load Data for the evaluation
data2evaluate = xutils.DataXRF().load_h5('F:/andrea/AIdataXRF/base/3314MargaretAnalysis_weighted_x1024_with_labels.h5')

# plt.figure()
# print(data2evaluate.data.reshape((-1, data2evaluate.data.shape[-1])).shape)
# plt.plot(data2evaluate._x, data2evaluate.data[0,0])

# plt.figure()
# data2evaluate = data2evaluate.resample(nbins=1024, bounds=(0,24))
# print(data2evaluate.data.reshape((-1, data2evaluate.data.shape[-1])).shape)
# plt.plot(data2evaluate._x, data2evaluate.data[0,0])

Loading: F:/andrea/AIdataXRF/base/3314MargaretAnalysis_weighted_x1024_with_labels.h5


In [5]:
# data2evaluate.metadata['labels'] = ['']
print(data2evaluate.data.shape)
# data2evaluate.labels = np.zeros((data2evaluate.shape[0],data2evaluate.shape[1], len(config['labels'])))
data2evaluate = data2evaluate.select_labels(config['labels'])
print('selected_labels:', data2evaluate.metadata['labels'])

(982, 708, 1024)
Adding empty labels: ['Ti-K']
selected_labels: ['Pb-L' 'Au-L' 'Ca-K' 'Co-K' 'Cu-K' 'Fe-K' 'Mn-K' 'Hg-L' 'Ti-K' 'Zn-K']


In [5]:
# sintetic_data_newtemplate = xutils.DataXRF().load_h5('F:/andrea/AIdataXRF/synthetic_data/SiO2_new_template_rl_1_50_data_1024.h5')
# sintetic_data = xutils.DataXRF().load_h5('F:/andrea/AIdataXRF/synthetic_data/MoSiO2_slfixed_rl_1_50_1024.h5')
# sintetic_data2 = xutils.DataXRF().load_h5('F:/andrea/AIdataXRF/synthetic_data/MoSiO2_slfixed_rl_1_90_1024.h5')

# Check the calibration of datasets

In [6]:
# amuleti_mean = np.sum(data2evaluate.data.reshape((-1, 1024)), axis=0)
# sintetic_mean = np.sum(sintetic_data.data.reshape((-1, 1024)), axis=0)
# sintetic_newtemplate_mean = np.sum(sintetic_data_newtemplate.data.reshape((-1, 1024)), axis=0)
# sintetic_mean2 = np.sum(sintetic_data2.data.reshape((-1, 1024)), axis=0)
# plt.figure()
# plt.plot(data2evaluate._x,amuleti_mean, '.-', label='amuleti')
# plt.plot(sintetic_data._x, sintetic_mean, '.-', label='1-50')
# plt.plot(sintetic_data2._x, sintetic_mean2, '.-', label='1-90', ms=10)
# plt.plot(sintetic_data_newtemplate._x, sintetic_newtemplate_mean, '.-', label='new template')
# plt.grid()
# plt.legend(frameon=True)
# plt.show()

In [7]:
# %matplotlib qt
# d2e = data2evaluate.data.reshape((-1,1024))
# for sidx, spec in enumerate(d2e):
#     if sidx % 100 ==0:
#         plt.plot(spec)

In [6]:
print(data2evaluate.labels[:,:,3].mean())

23.452210408827828


In [7]:
# print(data2evaluate.labels.shape)
data = torch.from_numpy(data2evaluate.data).reshape(-1,1,data2evaluate.data.shape[-1]).float()
labels = torch.from_numpy(data2evaluate.labels).reshape(-1,data2evaluate.labels.shape[-1]).float()
ds = dataset.SpectraDataset(data=data, labels=labels)
checkpoint = torch.load(model_path)
# print(checkpoint['model_state_dict']['fc.0.weight'].shape, model.fc)
model.load_state_dict(checkpoint['model_state_dict'], strict=False)
model.eval()

CNN2(
  (conv_0): Conv1d(1, 8, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (conv_1): Conv1d(8, 16, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (conv_2): Conv1d(16, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (conv_3): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (c0): Sequential(
    (0): Conv1d(1, 8, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Conv1d(8, 8, kernel_size=(1,), stride=(2,), bias=False)
  )
  (c1): Sequential(
    (0): Conv1d(8, 16, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Conv1d(16, 16, kernel_size=(1,), stride=(2,), bias=False)
  )
  (c2): Sequential(
    (0): Conv1d(16, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Conv1d(32, 32, kernel_size=(1,), stride=(2,), bias=False)
  )
  (c3): Sequential(
    (0): Conv

In [8]:
evaluate = DataLoader(ds,
                batch_size = config['batch_size'],
                shuffle = False,
                drop_last = False)
evaluate.shape = data2evaluate.labels.shape

In [9]:
print(data2evaluate.data.shape)

(982, 708, 1024)


In [10]:
image = []
device = torch.device(config['device'])
model.to(device)
for i,batch in enumerate(evaluate):

    x,y = batch

    x = x.to(device)
    y = y.to(device)

    outputs = model(x)
    image += [outputs.cpu().detach().numpy()]

image = np.vstack(image)
image = image.reshape(evaluate.shape)
print('\n',image.shape)


 (982, 708, 10)


In [11]:
%matplotlib qt
labels = data2evaluate.labels
image = image.astype(float)
cmap = 'jet'
print(data2evaluate.metadata['labels'])
for i, element in enumerate(data2evaluate.metadata['labels']):
    plt.figure()
    m = np.maximum(np.max(image[:,:,i]), np.max(labels[:,:,i]))

    plt.title('model '+ element)
    plt.imshow(image[:,:,i], cmap=cmap)
    plt.show()


# el = 1
# plt.figure()
# plt.imshow(image[:,:,el], cmap=cmap)
# plt.title('model '+ data2evaluate.metadata['labels'][el])


['Pb-L' 'Au-L' 'Ca-K' 'Co-K' 'Cu-K' 'Fe-K' 'Mn-K' 'Hg-L' 'Ti-K' 'Zn-K']


In [24]:
%matplotlib qt
labels = data2evaluate.labels
image = image.astype(float)
cmap = 'pink'
print(data2evaluate.metadata['labels'])
for i, element in enumerate(data2evaluate.metadata['labels']):
    plt.figure()
    ax_model = plt.subplot(221)

    ax_model.set_title('model '+ element)
    ax_model.imshow(image[:,:,i], cmap=cmap)
    ax_maxrf = plt.subplot(222)
    ax_maxrf.set_title('label '+ element)
    ax_maxrf.imshow(labels[:,:,i], cmap=cmap)
    
    m = np.maximum(np.max(image[:,:,i]), np.max(labels[:,:,i]))
    ax_hist = plt.subplot(212)
    ax_hist.set_title('histogram of counts')
    labelsy,labelsx = np.histogram((labels[:,:,i]).reshape(-1), bins=int((m+1)),range=(0, m+1))
    imagey,imagex = np.histogram(image[:,:,i].reshape(-1), bins=int((m+1)),range=(0, m+1))
    ax_hist.semilogy(labelsx[:-1]*1.45, labelsy*1.45, label=element +' - labels')
    ax_hist.semilogy(imagex[:-1], imagey, label=element +' - model', alpha=0.8)
    ax_hist.legend()
    plt.show()


# el = 1
# plt.figure()
# plt.imshow(image[:,:,el], cmap=cmap)
# plt.title('model '+ data2evaluate.metadata['labels'][el])


['Pb-L' 'Au-L' 'Ca-K' 'Co-K' 'Cu-K' 'Fe-K' 'Mn-K' 'Hg-L' 'Ti-K' 'Zn-K']


# save outputimages


In [ ]:
# save outputimages
outfilename = data2evaluate.metadata['path'].replace('\\', '/').split('/')[-3]
print(outfilename)

for i, element in enumerate(data2evaluate.metadata['labels']):
    plt.figure()
    plt.imshow(image[:,:,i], cmap=cmap)
    plt.title('model '+ element)
    tifffile.imwrite('F:/andrea/AIdataXRF/amuleti/outputs/'+outfilename+'/'+ element+'.tiff', image[:,:,i], 'imagej')
    # plt.imshow(tifffile.imread('F:/andrea/AIdataXRF/amuleti/outputs/'+outfilename+'_'+ element+'.tiff'))

Febbraio2022NuovaCassettaAmuleti1_A


c:\Python38\lib\site-packages\tifffile\tifffile.py:1455: UserWarning: TiffWriter: ignoring 'shape' argument
  warnings.warn(


In [ ]:
# data = xutils.DataXRF().load_h5('F:/andrea/AIdataXRF/synthetic_data/single_layer_rl_5_50_Mo_data_1024.h5').data[0]

# # for s, spe in enumerate(data):
# #     plt.plot(spe)
# #     print(np.min(spe))

In [1194]:
# runs = glob('./run/synth_scales0,5-1,5_*_CNN2MSELoss16_ch_run_0/')
# print (runs)
# images = []
# for run in runs:
#     base = run
#     model_path = glob(base+'*.pth')[-1]
#     print(model_path)
#     with open(base+'config.yaml','r') as file:
#         config = yaml.load(file,Loader=yaml.FullLoader)
#     model = nets.CNN2(channels=config['channels'], n_outputs=10)

#     #load Data for the evaluation
#     data2evaluate = xutils.DataXRF().load_h5('F:/andrea/AIdataXRF/base/ElGrecoAnalysis_weighted_x1024_with_labels.h5')
#     data2evaluate = data2evaluate.select_labels(config['labels'])


#     print(data2evaluate.labels.shape)
#     data = torch.from_numpy(data2evaluate.data).reshape(-1,1,data2evaluate.data.shape[-1]).float()
#     labels = torch.from_numpy(data2evaluate.labels).reshape(-1,data2evaluate.labels.shape[-1]).float()
#     ds = dataset.SpectraDataset(data=data, labels=labels)
#     checkpoint = torch.load(model_path)
#     model.load_state_dict(checkpoint['model_state_dict'], strict=False)
#     model.eval()


#     evaluate = DataLoader(ds,
#                     batch_size = config['batch_size'],
#                     shuffle = False,
#                     drop_last = False)
#     evaluate.shape = data2evaluate.labels.shape


#     image = []
#     device = torch.device(config['device'])
#     model.to(device)
#     for i,batch in enumerate(evaluate):

#         x,y = batch

#         x = x.to(device)
#         y = y.to(device)

#         outputs = model(x)
#         image += [outputs.cpu().detach().numpy()]

#     image = np.vstack(image)
#     print('\n',image.shape)
#     image = image.reshape(evaluate.shape)

#     images.append(image)

In [1195]:
# names = ['1-30only', '1-90only', '30-90only', 'uboundonly']
# for el, element in enumerate(data2evaluate.metadata['labels']):
#     plt.figure()
#     plt.title('histogram of counts - '+element)
#     # ax_hist = plt.subplot(111)
#     labelsy,labelsx = np.histogram(labels[:,el].reshape(-1), bins=int((8000+1)),range=(0, 8000+1))
#     plt.semilogy(labelsx[:-1], labelsy, label='labels')
#     for idx, img in enumerate(images):
#         img = img.astype(int)
        
#         imagey,imagex = np.histogram(img[:,:,el].reshape(-1), bins=int((8000+1)),range=(0, 8000+1))

#         plt.semilogy(imagex[:-1], imagey, label=names[idx], alpha=0.8)

#         plt.legend()
# plt.show()